In [ ]:
!pip install torch transformers pandas
!pip install scikit-learn

In [6]:



from transformers import BertTokenizer, BertModel, BertForSequenceClassification, pipeline, Trainer, TrainingArguments
import torch 
import pandas as pd
from sklearn.model_selection import train_test_split
import re


In [15]:
data_bancodobrasil = pd.read_json("data/banco_brasil.json")
data_bancodobrasil_con = pd.DataFrame({'texto': data_bancodobrasil["title"] + " " + data_bancodobrasil["description"]})
data_bancodobrasil_con['texto'] = data_bancodobrasil_con['texto'].str.replace('\n', '')

In [16]:
tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    return predicted_class, probabilities

In [19]:
data_bancodobrasil_con['sentiment'] = data_bancodobrasil_con['texto'].apply(lambda x: predict_sentiment(x)[0])
data_bancodobrasil_con['probabilities'] = data_bancodobrasil_con['texto'].apply(lambda x: predict_sentiment(x)[1].tolist())

data_bancodobrasil_con.to_csv('output.csv')